In [ ]:
import numpy as np
import cv2
import os

 
images_folder = './N2-embryo-10um-SRx5-crop-images-correct-angle-folders'
seg_folder = './N2-embryo-10um-SRx5-crop-seg-correct-angle-folders'

sub_folders = os.listdir(images_folder)

print(sub_folders)

pointcloud_folder = "N2-embryo-10um-SRx5-3D-pointcloud"


try:
    os.makedirs(pointcloud_folder)
    print(f"Folder '{pointcloud_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{pointcloud_folder}' already exists.")   



In [ ]:
def compute_np_iu(seg, gt):
    intersection = (seg & gt).astype(np.float32).sum()
    union = (seg | gt).astype(np.float32).sum()
    return intersection, union

def compute_np_iou(seg, gt):
    intersection, union = compute_np_iu(seg, gt)
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou

In [ ]:
import math
import open3d
from copy import deepcopy


for sub_folder in sub_folders:

    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)


    
    
    
    
    
    print(sub_folder)

    
    best_match_index = 0
    best_match_score = 0
    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            if image_index==0:

                img_tempate = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

            elif image_index>350:
                img = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                res = cv2.matchTemplate(img, img_tempate, cv2.TM_CCOEFF_NORMED)

                score = res[0][0]
                if score > best_match_score:
                    best_match_index = image_index
                    best_match_score = score
                    

    print('best_match_index = ',best_match_index, ', best_match_score = ' ,best_match_score)


    
    cloud_points = None
    degree_per_frame = 360.0/(best_match_index+1)

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= best_match_index:

                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                h,w = mask.shape


                
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                x,y,w,h = cv2.boundingRect(contours[0])

                rotate_axis = w/2-1

                rotate_axis = rotate_axis+x


                theta = 1.0*math.radians((image_index)*degree_per_frame)
                contour_points = np.zeros((contours[0].shape[0],3))
                contour_points[:,0:2] = contours[0].reshape(-1,2)

                
                new_contours = contour_points[contour_points[:,0]<241]
                

                center_y = y+(w/2)
                delta_y = 360-center_y
                

                pcd = open3d.geometry.PointCloud()
                pcd.points = open3d.utility.Vector3dVector(new_contours)

                R = pcd.get_rotation_matrix_from_xyz((0, theta, 0))
                pcd.rotate(R,center=(rotate_axis, 0, 0)) 

                if cloud_points == None:
                    cloud_points = pcd
                else:

                    cloud_points.paint_uniform_color([1,1,1])

                    pcd.paint_uniform_color([1,0,0])
                    cloud_points += pcd

    
    open3d.io.write_point_cloud(pointcloud_folder+'/'+sub_folder+'.pcd', cloud_points , write_ascii=True)  


    

    
    mean_iou = []
    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= best_match_index and image_index % int(best_match_index/12) == 0:
                
                img  = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 
                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 



                
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                offset_x,offset_y,w,h = cv2.boundingRect(contours[0])
                
                
                rotate_axis = w/2-1

                rotate_axis = rotate_axis+offset_x

                pcd2 = deepcopy(cloud_points)
                theta = -1.0*math.radians((image_index)*degree_per_frame)
                R = pcd2.get_rotation_matrix_from_xyz((0, theta, 0))
                pcd2.rotate(R, center=(rotate_axis, 0, 0))
                
                projection_90_degree = []
                for point in pcd2.points:
                    x, y, z = point
                    projection_90_degree.append([x,y])
                    
                projection = np.zeros_like(mask)
                for point in projection_90_degree:
                    x,y = point
                    x = int(x)
                    y = int(y)
                    try:
                        projection[y,x] = 255
                    except:
                        pass

                contours = cv2.findContours(
                    projection.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                cv2.fillPoly(projection, [contours[0]], (255))    

                iou = compute_np_iou(projection, mask)
                
                mean_iou.append(iou)
                
                
                
    print('mean_iou = ',np.mean(np.array(mean_iou)))

In [ ]:
10th_
best_match_index =  476 , best_match_score =  0.7352672
mean_iou =  0.9531676786830495
17th_
best_match_index =  572 , best_match_score =  0.7615381
mean_iou =  0.8714405119063917
1stc2_
best_match_index =  563 , best_match_score =  0.9253068
mean_iou =  0.9425399910021327
1stc3_
best_match_index =  239 , best_match_score =  0.7753495
mean_iou =  0.9599413648366988
2nd_
best_match_index =  534 , best_match_score =  0.8987711
mean_iou =  0.9287934110593469
3rd_
best_match_index =  707 , best_match_score =  0.8809713
mean_iou =  0.9499477233248937
4th_
best_match_index =  590 , best_match_score =  0.61554927
mean_iou =  0.8968795461722852
5th_
best_match_index =  588 , best_match_score =  0.89646286
mean_iou =  0.8314868990394546
6th_
best_match_index =  688 , best_match_score =  0.88626826
mean_iou =  0.8869941908428649
8th_
best_match_index =  482 , best_match_score =  0.91437435
mean_iou =  0.9688361468290139
9th_
best_match_index =  275 , best_match_score =  0.739135
mean_iou =  0.9009466009668012